# Introduction


**What?** Genetic Algorithm from scratch



# Import modules

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from collections import namedtuple

# Genetic Algorithm


- GA algorithms are a class of methods that try to mimic the way genes are passed down to future generation.
- A bunch of "answer candidates" and there is a feedback to figure out how close the candidate is to the "optimal" solution. 
- Far-from-optimal candidates gets dropped and are never seen again, while close-to-optimal candidates are combined with each other and maybe mutate slightly to see if they can get closer to optimal. 
- The mutation is an attempt to prevent the solution from getting stuck at the local optima. 
- **Chromosomes** is the representation of a solution candidate. They mate and mutate to produce offspring. They either die due to survival of the fittest, (natural selection).
- The collection of chromosomes is referred to as our population.



# Example


- Here is our problem: find a list of `N` integer numbers that equal `X` when summed together. 
- If we set `N = 5` and `X = 200` we then have:



In [4]:
chromo_size = 5
# We want our solution to be bounded between 0 to 100 (inclusive)
low  = 0 
high = 100

# This is an examlep of a chromosomes
np.random.randint(low, high + 1, chromo_size)

array([58, 89,  4, 30, 27])

In [5]:
pop_size = 6

# This is an example of population
pop = np.random.randint(low, high + 1, (pop_size, chromo_size))
pop

array([[33,  8, 73,  4, 88],
       [88, 33, 64, 19, 22],
       [ 5, 13,  7, 30, 89],
       [30, 63, 75, 65, 63],
       [41, 67, 61, 80, 65],
       [14, 24, 55, 87,  2]])

# Cost Function


- In this case, we might define a cost function to be the absolute difference between the sum of the and the target number X. The reason we're using the square of the difference is so that we never end up with a negative cost, you can choose to square it if you want to.
- That is a bit different than the case where we need the gradient. In that case the square is used not to have a derivative equal zero.



In [6]:
# determine the cost of a chromosome. lower is better
target = 200
cost = np.abs(np.sum(pop, axis = 1) - target)

# combine the cost and chromosome into one list
graded = [(c, list(p)) for p, c in zip(pop, cost)]
for cost, chromo in graded:
    print("chromo {}'s cost is {}".format(chromo, cost))

chromo [33, 8, 73, 4, 88]'s cost is 6
chromo [88, 33, 64, 19, 22]'s cost is 26
chromo [5, 13, 7, 30, 89]'s cost is 56
chromo [30, 63, 75, 65, 63]'s cost is 96
chromo [41, 67, 61, 80, 65]'s cost is 114
chromo [14, 24, 55, 87, 2]'s cost is 18


# Evolution - Crossover + Mutation


- Just like in evolution, you might be inclined to have the best and strongest chromosomes of the population mate with each other (The technical term for mating is **crossover**), with the hope that their offspring will be even healthier than either parent.
- Crossover is the main step of how you get from one generation of chromosomes to the next, but it alone has a problem: If all you do is mate your candidates to go from generation to generation, you'll have a chance of getting stuck near a "local optimum. Crossover helps discover more optimal solutions from already-good solutions, but it's the **mutation** that pushes the search for solutions in new directions. Mutation is a completely random process. 



# Implementing a GA algorithm


1. Randomly initialise a population
2. Calculate the cost/fitness score for each chromosomes.
3. Sort the chromosomes by the user-defined cost/fitness score.
4. Retain a certain number of the parent chromosomes
5. Mate the retained parent chromosomes to generate the children chromosomes.
6. Mutate the children chromosomes at random.
7. Compare the parent chromosomes and the children chromosomes and choose the best ones 
8. If the algorithm has not met some kind of completion criteria, return to step 2 with the new chromosomes. 



## Creating a GA class

In [9]:
class GA:
    """Genetic Algorithm 
    Parameters
    ----------
    generation : int
        number of iteration to train the algorithm
    pop_size : int
        number of chromosomes in the population
    chromo_size : int
        number of possible values (genes) per chromosome
    low, high : int
        lower_bound and upper_bound possible value of the randomly generated chromosome
    retain_rate : float 0 ~ 1
        the fraction of the best chromosome to retain. used to mate
        the children for the next generation
    mutate_rate : float 0 ~ 1
        the probability that each chromosome will mutate
    """
    
    def __init__( self, generation, pop_size, chromo_size, low, high, 
                  retain_rate, mutate_rate ):
        self.low  = low
        self.high = high
        self.pop_size = pop_size
        self.chromo_size = chromo_size
        self.generation  = generation
        self.retain_len  = int(pop_size * retain_rate)
        self.mutate_rate = mutate_rate
        self.info = namedtuple( 'info', ['cost', 'chromo'] )

    def fit(self, target):
        """
        target : int
            the targeted solution
        """

        # randomly generate the initial population, and evaluate its cost
        array_size = self.pop_size, self.chromo_size
        pop = np.random.randint(self.low, self.high + 1, array_size)
        graded_pop = self._compute_cost( pop, target )

        # store the best chromosome and its cost for each generation,
        # so we can get an idea of when the algorithm converged
        self.generation_history = []
        for _ in range(self.generation):
            graded_pop, generation_best = self._evolve(graded_pop, target)
            self.generation_history.append(generation_best)

        self.best = self.generation_history[self.generation - 1]
        self.is_fitted = True
        return self

    def _compute_cost(self, pop, target):
        """
        combine the cost and chromosome into one list and sort them
        in ascending order
        """
        cost = np.abs( np.sum(pop, axis = 1) - target )
        graded = [ self.info( c, list(p) ) for p, c in zip(pop, cost) ]
        graded = sorted(graded)
        return graded

    def _evolve(self, graded_pop, target):
        """
        core method that does the crossover, mutation to generate
        the possibly best children for the next generation
        """

        # retain the best chromos (number determined by the retain_len)
        graded_pop = graded_pop[:self.retain_len]
        parent = [ p.chromo for p in graded_pop ]

        # generate the children for the next generation 
        children = []
        while len(children) < self.pop_size:
            child = self._crossover(parent)
            child = self._mutate(child)
            children.append(child)

        # evaluate the children chromosome and retain the overall best,
        # overall simply means the best from the parent and the children, where
        # the size retained is determined by the population size
        graded_children = self._compute_cost(children, target)
        graded_pop.extend(graded_children)
        graded_pop = sorted(graded_pop)
        graded_pop = graded_pop[:self.pop_size]

        # also return the current generation's best chromosome and its cost
        generation_best = graded_pop[0]
        return graded_pop, generation_best 

    def _crossover(self, parent):
        """
        mate the children by randomly choosing two parents and mix 
        the first half element of one parent with the later half 
        element of the other
        """
        index1, index2 = random.sample( range(self.retain_len), k = 2 )
        male, female = parent[index1], parent[index2]
        pivot = len(male) // 2
        child = male[:pivot] + female[pivot:]
        return child


    def _mutate(self, child):
        """
        randomly change one element of the chromosome if it
        exceeds the user-specified threshold (mutate_rate)
        """
        if self.mutate_rate > random.random():
            idx_to_mutate = random.randrange(self.chromo_size)
            child[idx_to_mutate] = random.randint(self.low, self.high)

        return child

In [10]:
# calls the Genetic Algorithm
ga1 = GA( 
    generation = 10, 
    pop_size = 50, 
    chromo_size = 5, 
    low = 0, 
    high = 100, 
    retain_rate = 0.5, 
    mutate_rate = 0.2
)
ga1.fit(target = 200)

In [12]:
# the best chromo and its cost during each generation (iteration)
ga1.generation_history

[info(cost=0, chromo=[15, 43, 34, 65, 43]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6]),
 info(cost=0, chromo=[7, 68, 28, 91, 6])]

In [13]:
# Get the best one (one of them)
ga1.best

info(cost=0, chromo=[7, 68, 28, 91, 6])

# References


- [Step-by-step guide on how to build a GA](https://github.com/ethen8181/machine-learning/blob/master/ga/ga.py)
- [link to source code](https://nbviewer.jupyter.org/github/ethen8181/machine-learning/blob/master/ga/ga.ipynb)
- [Blog: Genetic Algorithms: Cool Name & Damn Simple](http://lethain.com/genetic-algorithms-cool-name-damn-simple/)
- [Blog: Machine Learning: Introduction to Genetic Algorithms](https://www.burakkanber.com/blog/machine-learning-genetic-algorithms-part-1-javascript/)
- [Tutorial: Applying a Genetic Algorithm to the traveling salesman problem](http://www.theprojectspot.com/tutorial-post/applying-a-genetic-algorithm-to-the-travelling-salesman-problem/5)

